# Homework 1
### Katherine Graham's Answers

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Jesse D. Jenkins and Michael R. Davidson (last updated: September 14, 2022)_

This Notebook will walk you through defining a simple transport flow model and then ask you to interact with the solutions and modify to model to add additional constraints...

## Setting up the model

### Load packages

In [194]:
using JuMP
using HiGHS
using DataFrames
using CSV

### Define sets

We will define two sets, both as arrays of strings

***Production plants, $P$***

In [195]:
P=["trenton", "newark"] # production plants

2-element Vector{String}:
 "trenton"
 "newark"

***Markets for products, $M$***

In [196]:
M=["newyork", "princeton", "philadelphia"] # markets for products

3-element Vector{String}:
 "newyork"
 "princeton"
 "philadelphia"

Note that sets can also be defined over intervals (as in `i=1:10`) or numerical vectors (as in `x=[2, 4, 5, 11]`) 

### Define parameters

We'll make use of the defined sets as indexes for our parameters...

***Plant production capacities***

In [197]:
plants = DataFrame(plant=P, capacity=[350,650])
# KATNOTE - Is this a pandas dataframe? 

,plant,capacity
,String,Int64
1,trenton,350
2,newark,650


***Demand for products***

Stored in a [DataFrame](https://juliadata.github.io/DataFrames.jl/stable/)

In [198]:
markets = DataFrame(
    market=M, 
    demand=[325, 300, 275]
)

,market,demand
,String,Int64
1,newyork,325
2,princeton,300
3,philadelphia,275


A few different ways to index into our DataFrames to access parameters (all of the below are equivalent)

In [199]:
plants[plants.plant.=="newark",:capacity] # option 1 # KATNOTE - THIS IS A PREFERRED METHOD FOR ME

1-element Vector{Int64}:
 650

In [200]:
plants[plants.plant.=="newark",:].capacity # option 2 # KATNOTE - THIS IS A PREFERRED METHOD FOR ME

1-element Vector{Int64}:
 650

In [201]:
plants.capacity[plants.plant.=="newark"] # option 3

1-element Vector{Int64}:
 650

In [202]:
plants[:,:capacity][plants.plant.=="newark"] # option 4

1-element Vector{Int64}:
 650

Note that DataFrame indexing returns an Array by default, in this case, a 1-element Array of type Int64 (64-bit integer), as indicated by `Array{Int64,1}` above. 

To access the single Int64 value, append `[1]` to any of the above to reference the first (and only) element in this array. 

In [203]:
plants.capacity[plants.plant.=="newark"][1]

650

In [204]:
typeof(plants.capacity[plants.plant.=="newark"][1])

Int64

In [205]:
typeof(plants.capacity[plants.plant.=="newark"])

Vector{Int64} (alias for Array{Int64, 1})

***Distance from plants to markets***

Stored in a JuMP [DenseAxisArray](https://jump.dev/JuMP.jl/v0.19/containers/) with data array and symbolic references across each of our sets (plants and markets), converted to Symbols for referencing

In [206]:
# two dimensional symbolic DenseAxisArray, with from/to distance pairs
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5;
     0.5 1.5 3.5],
    Symbol.(P),
    Symbol.(M),
)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:trenton, :newark]
    Dimension 2, [:newyork, :princeton, :philadelphia]
And data, a 2×3 Matrix{Float64}:
 2.5  0.5  1.5
 0.5  1.5  3.5

A couple example references to our DenseAxisArray to access parameters...

In [207]:
distances[:trenton, :newyork] #example of distance references

2.5

In [208]:
distances[:newark, :newyork] #example of distance references

0.5

In [209]:
distances[Symbol(P[2]),Symbol(M[1])] # another way to find distance from newark to trenton

0.5

In [210]:
distances[Symbol("newark"), Symbol("newyork")] # and a third...

0.5

***Costs of transport***

In [211]:
freight_cost = 90 # Cost of freight shipment per unit of distance

90

### Create model
(and specify the HiGHS solver)

In [212]:
transport = Model(HiGHS.Optimizer);

### Define variables

***Quantities of product to transport from plant $p \in P$ to market $m \in M$***

In [213]:
@variable(transport, X[P,M] >= 0) # non-negativity constraints

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia"]
And data, a 2×3 Matrix{VariableRef}:
 X[trenton,newyork]  X[trenton,princeton]  X[trenton,philadelphia]
 X[newark,newyork]   X[newark,princeton]   X[newark,philadelphia]

Example reference to single quantity decision variable, the quantity shipped from Newark to Philadelphia:

In [214]:
X["newark","philadelphia"]

X[newark,philadelphia]

### Define constraints

***Supply capacity constraint***

In [215]:
@constraint(transport, cSupply[p in P], 
    sum(X[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1]) # .= for elementwise? 

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["trenton", "newark"]
And data, a 2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cSupply[trenton] : X[trenton,newyork] + X[trenton,princeton] + X[trenton,philadelphia] ≤ 350.0
 cSupply[newark] : X[newark,newyork] + X[newark,princeton] + X[newark,philadelphia] ≤ 650.0

***Demand balance constraint***

Ensure all demand is satisfied at each market

In [216]:
@constraint(transport, cDemand[m in M], 
    sum(X[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1]) ### KATNOTE/Q : why is it indexed like this..

# KATNOTE -- THIS SYNTAX DOES NOT WORk
# @constraint(transport, cDemand[m in M], 
#     sum(X[p,m] for p in P) 
#     >= markets.demand[m][1]) ### KATNOTE/Q : why is it indexed like this..

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["newyork", "princeton", "philadelphia"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cDemand[newyork] : X[trenton,newyork] + X[newark,newyork] ≥ 325.0
 cDemand[princeton] : X[trenton,princeton] + X[newark,princeton] ≥ 300.0
 cDemand[philadelphia] : X[trenton,philadelphia] + X[newark,philadelphia] ≥ 275.0

### Define objective function

Minimize total cost of transport to satisfy all demand.

First we'll define an expression for total cost of shipments...

In [217]:
@expression(transport, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X[p,m] 
        for p in P, m in M)
    )

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

Now we'll minimize this total cost

In [218]:
@objective(transport, Min, eCost)

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

## Interact with the model

**(a)** Now let's solve the model. In the blank cell below, enter the command for JuMP to solve a model and run the cell

In [219]:
optimize!(transport)
print(transport)
# OPTIMAL VALUE = 85500

Presolving model
5 rows, 6 cols, 12 nonzeros
5 rows, 6 cols, 12 nonzeros
Presolve : Reductions: rows 5(-0); columns 6(-0); elements 12(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(900) 0s
          4     8.5500000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  8.5500000000e+04
HiGHS run time      :          0.00


**(b)** You've got a solution. Now query the objective function in the empty cell below and save it to a variable (name of your choice)

In [220]:
# Katnote - I couldn't find a good idea of what "query" means other than printing it
objfn_transport = objective_function(transport)

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

**(c)** Now query and save the optimal solution for X (the decisions about shipment quantities from plant to market) to an Array or DataFrame

In [221]:
# X
#solution_summary(transport)
# JuMP.value(X) # Incorrect syntax
solution_transport = JuMP.value.(X)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia"]
And data, a 2×3 Matrix{Float64}:
   0.0   75.0  275.0
 325.0  225.0    0.0

**(d)** Please interpret your results by writing an explanation in the markdown cell below. 

Which facility or facilities supplies the most demand in New York? Does this result make sense? Why?

Which facility or facilities supplies the most demand in Philadelphia? Does this result make sense? Why?

Which facility or facilities supplies the demand in Princeton? Does this result make sense? Why?

[ANSWER: ]

Newark supplies most of New York's demand with 325.0 (Trenton does not supply any of New York's demand). This makes sense because Newark is closer to New York than Trenton is, so the optimization will want to minimize transport costs by having Newark supply New York's demand. 
Similarly, Trenton is closer to Philly than Newark, so Trenton supplies all of Philadelphia's demand (275). 
Newark supplies most of Princeton's demand, with Trenton still supplying some of that demand. This makes sense because Trenton and Newark are both relatively close to Princeton, with Trenton being closer, *but* travel costs from Trenton to Princeton or from Newark to Princeton are the smallest costs as compared to Trenton to NY or Newark to Philly, and Newark has 300 more units of production capacity than Trenton did, so it ended up supplying most of Princeton's demand once the remainin supply from Trenton was used up. 


**(e)** A new market in New Brunswick appears, with a demand for 50 units. It is located 1.0 units away from both plants. Add this market to the model and solve again.

In [222]:
# PARAMETERS 
P=["trenton", "newark"] # production plants, redundent
M=["newyork", "princeton", "philadelphia", "newbrunswick"] # markets for products

plants = DataFrame(plant=P, capacity=[350,650])
markets = DataFrame(
    market=M, 
    demand=[325, 300, 275, 50]
)
# two dimensional symbolic DenseAxisArray, with from/to distance pairs
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5 1.0;
     0.5 1.5 3.5 1.0],
    Symbol.(P),
    Symbol.(M),
)
freight_cost = 90 # Cost of freight shipment per unit of distance


90

In [223]:
# Model
transport2 = Model(HiGHS.Optimizer);

# non-negativity constraints
@variable(transport2, Y[P,M] >= 0) 

# capacity constraints
@constraint(transport2, cSupply[p in P], 
    sum(Y[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1]) 

# demand constraints
@constraint(transport2, cDemand[m in M], 
    sum(Y[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1])

# objective function = eCost
@expression(transport2, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*Y[p,m] 
        for p in P, m in M)
    )

# definte the objective function as eCost (above)
@objective(transport2, Min, eCost)

# Run the model and print the optimal value
optimize!(transport2)
print(transport2)
# OPTIMAL VALUE = 90000



Presolving model
6 rows, 8 cols, 16 nonzeros
6 rows, 8 cols, 16 nonzeros
Presolve : Reductions: rows 6(-0); columns 8(-0); elements 16(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4(950) 0s
          5     9.0000000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  9.0000000000e+04
HiGHS run time      :          0.00


**(f)** What is new optimal solution? 

In [224]:
solution_transport2 = JuMP.value.(Y)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia", "newbrunswick"]
And data, a 2×4 Matrix{Float64}:
   0.0   75.0  275.0   0.0
 325.0  225.0    0.0  50.0

**(g)** Interpret this result in the markdown cell below. Which facility or facilities supplies the demand in New Brunswick? Does this result make sense? Why?

First, the objective value went up from 85,500 to 90,000, which makes sense because we will add to the total cost if we supply another city. 
Secondly, the solutions shows that 50 units of capacity are supplied to New Brunswick from Newark (despite Newark being further away than Trenton) because we had already used up Trenton's capacity when supplying other cities to minimize overall transportation costs.